# 정차시간 데이터셋 품질 검사

In [6]:
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'Malgun Gothic'

csv = '../../dataset/train/holding/240930_노선1개_3.4~9.23_평일_속도X/정차_train.csv'

# TIME_GAP 이상치 PLOTLY


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def show_time_gap_outliers_plotly(df):
    # df = df[df['TIME_GAP'] < 500000]
    df['TIME_GAP_MINUTES'] = df['TIME_GAP'] / 60

    Q1 = df['TIME_GAP'].quantile(0.25)
    Q3 = df['TIME_GAP'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # upper_bound = 300  # 상위 이상치 기준을 300초로 수동 설정

    lower_outliers = df[df['TIME_GAP'] < lower_bound]
    upper_outliers = df[df['TIME_GAP'] > upper_bound]

    outliers = df[(df['TIME_GAP'] < lower_bound) | (df['TIME_GAP'] > upper_bound)]
    print(f"이상치 개수: {len(outliers)}")
    print("\n이상치 상위 20개:")
    print(outliers.sort_values('TIME_GAP', ascending=False).head(20))

    print(f"상하위 이상치 기준점:")
    print(f" - Lower Bound (하위 이상치 기준점): {lower_bound:.2f}")
    print(f" - Upper Bound (상위 이상치 기준점): {upper_bound:.2f}")

    print(f"\n이상치 개수:")
    print(f" - 상위 이상치 개수 (Upper Outliers): {len(upper_outliers)}")
    print(f" - 하위 이상치 개수 (Lower Outliers): {len(lower_outliers)}")

    print("\n하위 이상치 데이터 (상위 5개):")
    print(lower_outliers.sort_values('TIME_GAP', ascending=True).head())

    # Plotly Box Plot 생성
    fig = go.Figure(
        data=[
            go.Box(y=df['TIME_GAP'], name="TIME_GAP", boxpoints='outliers')
        ]
    )

    fig.update_layout(
        title='TIME_GAP 분포와 이상치 (Plotly)',
        yaxis_title='TIME_GAP (초)'
    )

    fig.show()

    # Plotly를 사용한 히스토그램 생성
    fig = go.Figure(data=[go.Histogram(x=df['TIME_GAP_MINUTES'],
                                       nbinsx=50,  # 막대의 개수
                                       autobinx=False,  # 자동 bin 설정 해제
                                       xbins=dict(start=0, end=50, size=1),  # 0~50분, 1분 간격
                                       marker_color='lightblue',
                                       opacity=0.75)])

    # 그래프 레이아웃 설정
    fig.update_layout(
        title='Distribution of TIME_GAP (Minutes)',
        xaxis_title='Time Gap (minutes)',
        yaxis_title='Frequency',
        bargap=0.1,  # 막대 사이의 간격
    )

    # x축 범위 설정 (0~50분)
    fig.update_xaxes(range=[0, 50])

    # 그래프 표시
    fig.show()
    

In [ ]:
# csv = '../../dataset/train/holding/240920/금요일/정차시간_4_5_금요일학습데이터.csv'
# csv = '../../dataset/inference/holding/240920/금요일/정차시간_4_5_금요일학습데이터.csv'
dtype_spec = {
    'DAY_TYPE': 'int8',
    'BUSROUTE_ID': 'str',
    'BUSSTOP_ID': 'str',
    'DEP_TIME': 'str',
    'TIME_GAP': 'int32'
}
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSSTOP_ID', 'DEP_TIME']
usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSSTOP_ID', 'DEP_TIME', 'TIME_GAP']

data_pd = pd.read_csv(csv, skipinitialspace=True, usecols=usecols, dtype=dtype_spec)

show_time_gap_outliers_plotly(data_pd)

# 필터링 파일 저장

In [4]:
import pandas as pd
import os

def filter_and_save_data(df, output_file='filtered_data.csv'):
    original_count = len(df)
    print(f"초기 데이터 행 수: {original_count}")

    # 2. 중복 행 제거
    filtered_df = df.drop_duplicates()
    removed_count_duplicates = original_count - len(filtered_df)
    print(f"중복된 {removed_count_duplicates}개의 행이 제거되었습니다. 남은 행 수: {len(filtered_df)}")
    
    # 3. TIME_GAP이 0이거나 120보다 큰 행 제외
    original_count = len(filtered_df)
    filtered_df = filtered_df[(filtered_df['TIME_GAP'] > 0) & (filtered_df['TIME_GAP'] <= 120)]
    removed_count_time_gap = original_count - len(filtered_df)
    print(f"정차시간(TIME_GAP)이 0미만 이거나 120초보다 큰 {removed_count_time_gap}개의 행이 제거되었습니다. 남은 행 수: {len(filtered_df)}")
    
    # 결과를 새로운 CSV 파일로 저장
    filtered_df.to_csv(output_file, index=False)
    print(f"필터링된 데이터를 '{output_file}'로 저장했습니다. 최종 행 수: {len(filtered_df)}")

In [5]:

# csv = '../../dataset/train/holding/240920/금요일/정차시간_4_5_금요일학습데이터.csv'
# dtype_spec = {
#     'DAY_TYPE': 'int8',
#     'BUSROUTE_ID': 'str',
#     'BUSSTOP_ID': 'str',
#     'DEP_TIME': 'str',
#     'TIME_GAP': 'int32'
# }
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSSTOP_ID', 'DEP_TIME', 'TIME_GAP']

# 원본 데이터를 불러온 후 함수 호출
data_pd = pd.read_csv(csv, skipinitialspace=True, usecols=usecols, dtype=dtype_spec)

# 결과 저장
input_filename = os.path.basename(csv)
output_filename = os.path.splitext(input_filename)[0] + '_filtered.csv'
result_file = os.path.join(os.path.dirname(csv), output_filename)


filter_and_save_data(data_pd, result_file)


초기 데이터 행 수: 749804
중복된 20609개의 행이 제거되었습니다. 남은 행 수: 729195
정차시간(TIME_GAP)이 0미만 이거나 120초보다 큰 0개의 행이 제거되었습니다. 남은 행 수: 729195
필터링된 데이터를 '../../dataset/train/holding/240927_7,8공휴일\정차_7,8공휴일_train_filtered.csv'로 저장했습니다. 최종 행 수: 729195


# 중복되는 행 찾기

In [4]:
# 중복되는 행 찾기
def find_duplicate_rows(df):
    duplicate_rows = df[df.duplicated()]
    return duplicate_rows
# csv = '../../dataset/train/holding/240920/금요일/정차시간_4_5_금요일학습데이터.csv'

df = pd.read_csv(csv, usecols=usecols, dtype=dtype_spec)
duplicate_rows = find_duplicate_rows(df)
duplicate_rows

print(df.columns)

Index(['DAY_TYPE', 'BUSROUTE_ID', 'BUSSTOP_ID', 'DEP_TIME', 'TIME_GAP'], dtype='object')


# Plotly를 사용한 TIME_GAP 히스토그램 생성